# Baseline 1-PH: Alpha complex e eccentricity lower-star

Confronto con i metodi biparametrici (CMD, MD).
Stessa generazione dati e stessa pipeline di classificazione del notebook attrattori.

In [32]:
import numpy as np
import gudhi as gd
from sklearn.metrics import pairwise_distances as pdist_sklearn
from sklearn.model_selection import train_test_split
from collections import Counter
from scipy.integrate import solve_ivp
from tqdm import tqdm
import time, warnings
warnings.filterwarnings('ignore')
np.random.seed(42)

In [33]:
# --- Parametri (stessi del notebook bifiltrazione) ---
n = 5000           # punti per sample
n_ex = 40         # sample per classe
BOTTLENECK_E = 0.01
K_KNN = 3

class_names = ['Lorenz', 'Rössler', 'Thomas', 'Sprott', 'Four-Wing']
n_classes = len(class_names)

In [34]:
# --- ODE (identico al notebook bifiltrazione) ---

def lorenz_rhs(t, s):
    x, y, z = s
    sigma, rho, beta = 10.0, 28.0, 8.0/3.0
    return [sigma*(y - x), x*(rho - z) - y, x*y - beta*z]

def rossler_rhs(t, s):
    x, y, z = s
    a, b, c = 0.2, 0.2, 5.7
    return [-(y + z), x + a*y, b + z*(x - c)]

def thomas_rhs(t, s):
    x, y, z = s
    b = 0.208186
    return [np.sin(y) - b*x, np.sin(z) - b*y, np.sin(x) - b*z]

def sprott_rhs(t, s):
    x, y, z = s
    a, b = 2.07, 1.79
    return [y + a*x*y + x*z, 1.0 - b*x**2 + y*z, x - x**2 - y**2]

def fourwing_rhs(t, s):
    x, y, z = s
    a, b, c = 0.2, 0.01, -0.4
    return [a*x + y*z, b*x + c*y - x*z, -z - x*y]

attractor_configs = [
    (lorenz_rhs,   20,  100, 5.0),
    (rossler_rhs,  50,  200, 1.0),
    (thomas_rhs,  100,  500, 1.0),
    (sprott_rhs,   50,  200, 0.5),
    (fourwing_rhs, 50,  300, 0.1),
]


def generate_attractor(cls_idx, n_pts=500, noise=0.0, seed=None):
    rhs, t_burn, t_run, x0_scale = attractor_configs[cls_idx]
    rng = np.random.RandomState(seed)
    x0 = rng.uniform(-x0_scale, x0_scale, 3)
    sol0 = solve_ivp(rhs, [0, t_burn], x0, rtol=1e-8, atol=1e-8, dense_output=True)
    y0 = sol0.y[:, -1]
    n_dense = max(10000, 20*n_pts)
    t_eval = np.linspace(0, t_run, n_dense)
    sol = solve_ivp(rhs, [0, t_run], y0, t_eval=t_eval, rtol=1e-8, atol=1e-8)
    traj = sol.y.T
    idx = np.linspace(0, len(traj)-1, n_pts, dtype=int)
    pts = traj[idx].copy()
    # Cubo unitario [0,1]^3
    pmin, pmax = pts.min(axis=0), pts.max(axis=0)
    rng_ = pmax - pmin
    rng_[rng_ < 1e-12] = 1.0
    pts = (pts - pmin) / rng_
    if noise > 0:
        pts += np.random.RandomState(seed).normal(0, noise, pts.shape)
    return pts

In [35]:
def safe_bottleneck(pd1, pd2, e=0.0):
    if len(pd1) == 0 and len(pd2) == 0: return 0.0
    if len(pd1) == 0: return float(np.max((pd2[:,1]-pd2[:,0])/2))
    if len(pd2) == 0: return float(np.max((pd1[:,1]-pd1[:,0])/2))
    return gd.bottleneck_distance(pd1, pd2, e)


def extract_finite_pd(st, dim):
    pd = st.persistence_intervals_in_dimension(dim)
    if pd is not None and len(pd) > 0:
        pd = pd[np.isfinite(pd[:, 1])]
    if pd is None or len(pd) == 0:
        pd = np.empty((0, 2))
    return pd


def classify_knn_most_confident(D_train_list, D_test_list,
                                labels_train, labels_test, k=3):
    labels_train = np.asarray(labels_train)
    labels_test = np.asarray(labels_test)
    n_degrees = len(D_train_list)
    n_test = D_test_list[0].shape[0]
    final = np.zeros(n_test, dtype=int)
    for i in range(n_test):
        best_conf, best_pred, best_dist = -1.0, 0, np.inf
        for d in range(n_degrees):
            dists_i = D_test_list[d][i]
            k_nearest = np.argsort(dists_i)[:k]
            nn_labels = labels_train[k_nearest]
            pred = Counter(nn_labels).most_common(1)[0][0]
            conf = np.sum(nn_labels == pred) / k
            avg_d = np.mean(dists_i[k_nearest])
            if conf > best_conf or (conf == best_conf and avg_d < best_dist):
                best_conf, best_pred, best_dist = conf, pred, avg_d
        final[i] = best_pred
    return float(np.mean(final == labels_test))

## Generazione dati

In [36]:
Data, Labels = [], []
for cls_idx in range(n_classes):
    for j in range(n_ex):
        Labels.append(cls_idx)
        Data.append(generate_attractor(cls_idx, n_pts=n,
                                       noise=0.0,
                                       seed=42 + cls_idx*1000 + j))
Labels = np.array(Labels)
N = len(Data)
print(f"N = {N} ({n_classes} classi × {n_ex} samples), n = {n} punti")

N = 200 (5 classi × 40 samples), n = 5000 punti


## 1. Alpha 1-PH

In [37]:
print("Computing Alpha 1-PH...")
t0 = time.time()

PDs_alpha = []
for i in tqdm(range(N)):
    ac = gd.AlphaComplex(points=Data[i])
    st = ac.create_simplex_tree()
    st.persistence()
    PDs_alpha.append((extract_finite_pd(st, 0), extract_finite_pd(st, 1)))

# Distance matrices
D_alpha_H0 = np.zeros((N, N))
D_alpha_H1 = np.zeros((N, N))
for i in tqdm(range(N), desc='Distances'):
    for j in range(i+1, N):
        d0 = safe_bottleneck(PDs_alpha[i][0], PDs_alpha[j][0], BOTTLENECK_E)
        d1 = safe_bottleneck(PDs_alpha[i][1], PDs_alpha[j][1], BOTTLENECK_E)
        D_alpha_H0[i,j] = D_alpha_H0[j,i] = d0
        D_alpha_H1[i,j] = D_alpha_H1[j,i] = d1

time_alpha = time.time() - t0
print(f"Done in {time_alpha:.0f}s")

Computing Alpha 1-PH...


Distances: 100%|██████████| 200/200 [00:02<00:00, 77.14it/s] 

Done in 109s


## 2. Eccentricity lower-star 1-PH

In [38]:
print("Computing Eccentricity lower-star 1-PH...")
t0 = time.time()

PDs_ecc = []
for i in tqdm(range(N)):
    # Eccentricity normalizzata a [0,1]
    ecc = np.max(pdist_sklearn(Data[i]), axis=1)
    emin, emax = ecc.min(), ecc.max()
    if emax > emin:
        ecc = (ecc - emin) / (emax - emin)

    # Alpha complex come supporto simpliciale, filtrazione lower-star da eccentricity
    ac = gd.AlphaComplex(points=Data[i])
    st = ac.create_simplex_tree()
    for simplex, _ in st.get_simplices():
        val = max(ecc[v] for v in simplex)
        st.assign_filtration(simplex, float(val))
    st.make_filtration_non_decreasing()
    st.persistence()
    PDs_ecc.append((extract_finite_pd(st, 0), extract_finite_pd(st, 1)))

# Distance matrices
D_ecc_H0 = np.zeros((N, N))
D_ecc_H1 = np.zeros((N, N))
for i in tqdm(range(N), desc='Distances'):
    for j in range(i+1, N):
        d0 = safe_bottleneck(PDs_ecc[i][0], PDs_ecc[j][0], BOTTLENECK_E)
        d1 = safe_bottleneck(PDs_ecc[i][1], PDs_ecc[j][1], BOTTLENECK_E)
        D_ecc_H0[i,j] = D_ecc_H0[j,i] = d0
        D_ecc_H1[i,j] = D_ecc_H1[j,i] = d1

time_ecc = time.time() - t0
print(f"Done in {time_ecc:.0f}s")

Computing Eccentricity lower-star 1-PH...


Distances: 100%|██████████| 200/200 [00:00<00:00, 2184.12it/s]

Done in 174s


## Classificazione 10-fold CV

In [39]:
def extract_distance_blocks(D_full, train_idx, test_idx):
    return D_full[np.ix_(train_idx, train_idx)], D_full[np.ix_(test_idx, train_idx)]

methods = {
    'Alpha 1-PH':       (D_alpha_H0, D_alpha_H1),
    'Ecc lower-star':   (D_ecc_H0, D_ecc_H1),
}

accs = {m: [] for m in methods}

for fold in range(10):
    idx_tr, idx_te, lab_tr, lab_te = train_test_split(
        np.arange(N), Labels, train_size=0.7, stratify=Labels, random_state=42+fold)
    for mname, (D0, D1) in methods.items():
        blocks = [extract_distance_blocks(D, idx_tr, idx_te) for D in [D0, D1]]
        accs[mname].append(classify_knn_most_confident(
            [b[0] for b in blocks], [b[1] for b in blocks],
            lab_tr, lab_te, K_KNN))

print('=' * 55)
print(f'{"Method":<20} {"Accuracy":<20} {"Time (s)":<15}')
print('=' * 55)
for mname in methods:
    v = accs[mname]
    t = time_alpha if 'Alpha' in mname else time_ecc
    print(f'{mname:<20} {np.mean(v):.2%} ± {np.std(v):.2%}     {t:.0f}')
print('=' * 55)
print(f'\nPer-fold details:')
for mname in methods:
    print(f'  {mname}: {[f"{v:.2%}" for v in accs[mname]]}')

Method               Accuracy             Time (s)       
Alpha 1-PH           47.17% ± 6.91%     109
Ecc lower-star       30.17% ± 5.40%     174

Per-fold details:
  Alpha 1-PH: ['51.67%', '35.00%', '45.00%', '51.67%', '41.67%', '41.67%', '50.00%', '55.00%', '58.33%', '41.67%']
  Ecc lower-star: ['30.00%', '28.33%', '23.33%', '28.33%', '21.67%', '33.33%', '38.33%', '38.33%', '33.33%', '26.67%']


In [40]:
np.save(f'Attractors_1ph_{n:03d}.npy', accs)